In [1]:
import numpy as np
import edward as ed
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
def build_toy_dataset(N):
    theta = [[0.1, 0.2, 0.7], [0.7,0.2,0.1], 
                      [0.2,0.7,0.1], [0.7,0.1,0.2]] # 4 Documents, 3 Topics
    phi = [[0.1,0.1,0.1,0.1,0.1,0.5], 
                    [0.1,0.1,0.1,0.1,0.5,0.1], 
                    [0.2,0.2,0.2,0.2,0.1,0.1], # 6 Words per topic
                   ]
    x = np.zeros((N*4, 2), dtype=np.int32)
    x[:,1] = [item for item,count in [(0,N), (1,N),(2,N),(3,N)] for i in range(count)]
    for n in range(x.shape[0]):
        k = np.argmax(np.random.multinomial(1, theta[x[n,1]])) # which topic depending on document
        x[n, 0] = np.argmax(np.random.multinomial(1, phi[k]))

    return x

D = 4  # number of documents
N = 100  # number of words per document

x_train = build_toy_dataset(N*D)

In [3]:
K = 3 # Number of topics
V = len(set(x_train[:,0])) # Unique words observed

In [4]:
x_train[95:104] # Word ID| Document ID

array([[0, 0],
       [0, 0],
       [5, 0],
       [5, 0],
       [5, 0],
       [1, 0],
       [4, 0],
       [4, 0],
       [1, 0]], dtype=int32)

The generative process of LDA is:
- 

In [5]:
from edward.models import Dirichlet, Categorical, ParamMixture

The sample of a k-dimensional Dirichlet distribution are k 'probabilities' that sum up to 1

In [6]:
with tf.Session() as sess:
    alpha = [1.,10,100]
    eta = Dirichlet(alpha, sample_shape =3 )
    print(eta.sample(1).eval())
    
    gamma = Categorical(probs = eta)
    print(gamma.sample(10).eval())

[[0.0041837  0.08712402 0.9086923 ]]
[[2 1 2]
 [2 2 2]
 [1 2 2]
 [2 2 2]
 [2 2 2]
 [2 2 1]
 [2 2 2]
 [2 1 2]
 [2 2 1]
 [2 2 2]]


In [7]:
with tf.Session() as sess:
    pi = Dirichlet([1.,1]) # Mixture of 2 distributions
    alpha = [[1.,1,100],[1.,100,1]] # The parameters for each of the 2 distributions
    eta = Dirichlet(alpha) # Draw a distribution from the 3 distr.
    print(pi)
    print(eta)
     
    gamma = ParamMixture(pi, {"probs":eta}, Categorical) # Draw a pi then draw a value from the distribution
    print(gamma.sample(20).eval())

RandomVariable("Dirichlet_1/", shape=(2,), dtype=float32)
RandomVariable("Dirichlet_2/", shape=(2, 3), dtype=float32)
[2 2 1 0 1 2 1 1 2 2 1 2 2 2 2 2 2 2 1 1]


In [8]:
from edward.models import Deterministic
d = tf.placeholder(tf.int32, [D*N,])
theta = Dirichlet(tf.ones(K), sample_shape = D) # Dir(alpha)
pi = tf.reshape(tf.gather(theta,d, axis=0), [3,N*D])

In [9]:
phi = Dirichlet(tf.ones(V), sample_shape = K) # Dir(beta)

In [10]:
with tf.Session() as sess:
    print(phi)

RandomVariable("Dirichlet_4/", shape=(3, 6), dtype=float32)


In [11]:
w = ParamMixture(pi, # The mixture parameter 
                 {'probs':phi}, # The parameters of the distribution
                 Categorical) #, # The distribution to sample from
                 #sample_shape = N) # The number of samples

In [12]:
z = w.cat

## Inference via Gibbs sampling

In [12]:
# Conditionals
theta_cond = ed.complete_conditional(theta)
phi_cond = ed.complete_conditional(phi)
w_cond = ed.complete_conditional(w)
z_cond = ed.complete_conditional(z)

sess = ed.get_session()

NotImplementedError: Conditional distribution has sufficient statistics (('#Log', ('#x',)), ('#Log', (<tf.Tensor 'Reshape:0' shape=(3, 400) dtype=float32>, (<tf.Tensor 'Gather:0' shape=(400, 3) dtype=float32>, ('#x',), (<tf.Tensor 'Placeholder:0' shape=(400,) dtype=int32>,)), (<tf.Tensor 'Reshape/shape:0' shape=(2,) dtype=int32>,)))), but no available exponential-family distribution has those sufficient statistics.

In [ ]:
# Initialize randomly
theta_est, phi_est, w_est, z_est = sess.run([theta, phi, w, z])

print('Initial parameters:')
print('pi:', theta_est)
print('mu:', phi_est)
print('sigmasq:', z_est)
print()

In [ ]:
# Gibbs sampler
cond_dict = {pi: pi_est, mu: mu_est, sigmasq: sigmasq_est,
           z: z_est, x: x_data}
t0 = time()
T = 500
for t in range(T):
z_est = sess.run(z_cond, cond_dict)
cond_dict[z] = z_est
pi_est, mu_est = sess.run([pi_cond, mu_cond], cond_dict)
cond_dict[pi] = pi_est
cond_dict[mu] = mu_est
sigmasq_est = sess.run(sigmasq_cond, cond_dict)
cond_dict[sigmasq] = sigmasq_est
print('took %.3f seconds to run %d iterations' % (time() - t0, T))

print()
print('Final sample for parameters::')
print('pi:', pi_est)
print('mu:', mu_est)
print('sigmasq:', sigmasq_est)
print()

## Inference via 

In [14]:
T = 100

In [15]:
from edward.models import Empirical

qtheta = Empirical(tf.get_variable(
"qtheta/params", [T, K], # Nr. samples x dimensionality of groups
initializer = tf.constant_initializer(1.0/K))) # Initialized with uniform mixture

qphi = Empirical(tf.get_variable(
"qphi/params", [T, V], # Nr of samples x Vocabulary size
initializer =  tf.constant_initializer(1.0/K))) 

qw = Empirical(tf.get_variable(
"qw/params", [T, N],
initializer = tf.ones_initializer(),
dtype=tf.int32))

qz =Empirical(tf.get_variable(
"qz/params", [T, N],
initializer = tf.zeros_initializer(),
dtype=tf.int32))

In [16]:
from edward.inferences import SGLD
inference = SGLD({theta: qtheta, phi: qphi, w: qw, z:qz}, 
                  data={w: x_train[:,0], 
                        d: x_train[:,1]})
inference.initialize

TypeError: Key-value pair in latent_vars does not have same shape: (4, 3), (3,)

In [17]:
from edward.inferences import Gibbs
inference = Gibbs({theta: qtheta, phi: qphi, w: qw, z:qz}, 
                  data={w: x_train[:,0], d: x_train[:,1]})
inference.initialize

NotImplementedError: Conditional distribution has sufficient statistics (('#Log', ('#x',)), ('#Log', (<tf.Tensor 'Reshape:0' shape=(3,) dtype=float32>, (<tf.Tensor 'Gather:0' shape=(1, 3) dtype=float32>, ('#x',), (<tf.Tensor 'Placeholder:0' shape=(1,) dtype=int32>,)), (<tf.Tensor 'Reshape/shape:0' shape=(1,) dtype=int32>,)))), but no available exponential-family distribution has those sufficient statistics.